In [1]:
import sys
sys.path.append('../')
from time import sleep
from models.attention_series import Grade_regressor
import spectral
from configs.training_cfg import device
import torch
import ast
import numpy as np
from PIL import Image
import pandas as pd
from spectral import imshow
from matplotlib.pyplot import MultipleLocator
import matplotlib.pyplot as plt

spectral.settings.envi_support_nonlowercase_params = True

pre_list = []
erro_list = []

In [2]:
# 测试样本绝对误差
for k in range(1):
    model = Grade_regressor().to(device).eval()
    model.load_state_dict(torch.load("..\\ckpt\\近红外3_48000.pt".format(k+1)))
    result = []
    err = 0
    err_count = 0 
    pool = torch.nn.AvgPool2d(3,3)
    mask_rgb_values = [[255,242,0],[34,177,76],[255,0,88]]

    spec_id = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,57,58,59,60,61]    # 选择评估的成像光谱图片编号


    for id in spec_id:
        img = spectral.envi.open("D:\\近红外部分\\spectral_data\{}-Radiance From Raw Data-Reflectance from Radiance Data and Measured Reference Spectrum.bip.hdr".format(id))

        # 根据模型使用波段选择
        # img_data = torch.Tensor(img.asarray()/6000)[:,:,:-4]
        img_data = torch.Tensor(img.asarray()/6000)[:,:,:]
        # img_data = pool(img_data.permute(1,2,0)).permute(2,0,1)
        mask = np.array(Image.open("D:\\近红外部分\\spectral_data\\{}-Radiance From Raw Data-Reflectance from Radiance Data and Measured Reference Spectrum.bip.hdr_mask.png".format(id)))
        if mask.shape[2] == 4:
            mask = mask[:,:,:-1]
        gt_TFe = ast.literal_eval(img.metadata['gt_TFe'])

        with torch.no_grad():
            img_data = pool(img_data.permute(2,0,1)).permute(1,2,0)
            row,col,_ = img_data.shape
            heat_map = []

            for i in range(row):
                heat_map.append(model(img_data[i].to(device)).squeeze(1).unsqueeze(0).to("cpu"))   # 只评估Tfe
                torch.cuda.empty_cache()

        heat_map = torch.cat(heat_map, dim=0)

        predict_sum = torch.Tensor([0.,0.,0.])
        pixel_count = torch.Tensor([0, 0, 0])
        gt = torch.Tensor(gt_TFe)

        values = [[],[],[]] # 分析数据分布

        for r in range(row):
            for c in range(col):
                for i in range(3):
                    if mask[r*3+1,c*3+1].tolist() == mask_rgb_values[i]:
                        predict_sum[i] += heat_map[r,c] 
                        values[i].append(heat_map[r,c])    # 分析数据分布
                        pixel_count[i] += 1

        prediction = predict_sum / pixel_count * 100

        err_list = ((prediction-gt)).tolist()
        pred_list = (prediction).tolist()
        for i in range(err_list.__len__()):
            erro_list.append(round(err_list[i],2))
            pre_list.append(round(pred_list[i], 2))
            

    # avg_err = err/(err_count)
    # print("{}k  avgmse:{}".format(k+1, avg_err))

RuntimeError: The size of tensor a (3) must match the size of tensor b (2) at non-singleton dimension 0

In [3]:
prediction.

tensor([25.6812, 21.4506,     nan])

In [ ]:
erro_list

[-2.53, -2.32, -4.47]

In [ ]:
pre_list

[43.78, 54.6, 41.08]